In [ ]:
import pandas as pd
import sklearn.linear_model

# import keras_deepCNN
# from tensorflow.keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
# from tensorflow.keras.models import Model, load_model
# from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import numpy as np
from sklearn.utils import shuffle
import time

# gradient boosting for classification in scikit-learn
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RepeatedStratifiedKFold
from matplotlib import pyplot
import sklearn.metrics as metrics
from sklearn.preprocessing import LabelEncoder 
from xgboost import XGBClassifier
from sklearn.externals import joblib

In [ ]:
# Load from file
joblib_file = "joblib_model_gradientBoosting_train70_dev36.pkl"
joblib_model = joblib.load(joblib_file)


In [ ]:
tournament_data = pd.read_csv("data/numerai_datasets_28.03.21/numerai_tournament_data.csv")
validation_data = tournament_data.loc[tournament_data.data_type == 'validation']

In [ ]:
prediction_data = tournament_data.loc[tournament_data.data_type == 'live']

feature_cols = prediction_data.columns[prediction_data.columns.str.startswith('feature')]
X_val = validation_data[feature_cols].reset_index().drop(['index'], axis = 1).to_numpy()
Y_val = validation_data.target.to_numpy()

X_pred = prediction_data[feature_cols].reset_index().drop(['index'], axis = 1).to_numpy()

y_val_pred = joblib_model.predict(X_val)
corVal = 0
allTrainSamples = len(y_val_pred)
for i in range(allTrainSamples):
    if y_val_pred[i] == Y_val[i]:
        corVal +=1
accuracy = corVal / allTrainSamples

print("validation accuracy = ", accuracy)

In [ ]:
X_tournament = tournament_data[feature_cols].reset_index().drop(['index'], axis = 1).to_numpy()

Y_pred = joblib_model.predict(X_tournament)

predictions_df = tournament_data["id"].to_frame()
predictions_df["prediction_kazutsugi"] = Y_pred
predictions_df.head()

predictions_df.to_csv("predictions.csv", index=False)

In [ ]:
import numerapi
public_id  = "5PKOLW4ZJQDSMTC2QWPPG2QEHB427MFJ"
secret_key = "I26YRNIBRQF47E6SO6VMLTLGN4O2MHL6ADUB4JNNQAYW3DPCH6JKY4HS5R2PYKLB"
model_id   = "e994d440-764d-495d-8d60-7dbac3ac615b"
napi = numerapi.NumerAPI(public_id=public_id, secret_key=secret_key)
submission_id = napi.upload_predictions("predictions.csv", model_id=model_id)

In [ ]:
predictions_df.prediction_kazutsugi.hist(bins=10)